In [ ]:
import pandas as pd
import quandl
import matplotlib.pyplot as plt
import numpy as np
import math
import time
from scipy import stats
%matplotlib inline

In [ ]:
def read_pickle(instrument, startdate, enddate, resample_period):
    path_ = ('/home/jake/Code/kaiseki/local_data/' + instrument + "_pickle")
    df = pd.read_pickle(path_).loc[startdate:enddate].resample(resample_period).last()
    return df


In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.grid_objs as gd
import seaborn as sns
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True, theme='polar')

In [ ]:
# Load data for 15 min model
instrument = ""
startdate = '2015-01-01'
enddate = '2018-12-31'
resample_period = "15t"


usdeur = read_pickle("EURUSD", startdate, enddate, resample_period)
usdeur= usdeur[['open','high','low','close']]
usdeur= 1/usdeur
# usdeur.columns = [['usdeur_open','usdeur_high','usdeur_low','usdeur_close']]

usdaud = read_pickle("AUDUSD", startdate, enddate, resample_period)
usdaud= usdaud[['open','high','low','close']]
usdaud= 1/usdaud
# usdaud.columns = [['usdaud_open','usdaud_high','usdaud_low','usdaud_close']]

usdcad = read_pickle("USDCAD", startdate, enddate, resample_period)
usdcad= usdcad[['open','high','low','close']]
# usdcad.columns = [['usdcad_open','usdcad_high','usdcad_low','usdcad_close']]

usdchf = read_pickle("USDCHF", startdate, enddate, resample_period)
usdchf= usdchf[['open','high','low','close']]
# usdchf.columns = [['usdchf_open','usdchf_high','usdchf_low','usdchf_close']]

usdjpy = read_pickle("USDJPY", startdate, enddate, resample_period)
usdjpy= usdjpy[['open','high','low','close']]
# usdjpy.columns = [['usdjpy_open','usdjpy_high','usdjpy_low','usdjpy_close']]

usdgbp = read_pickle("GBPUSD", startdate, enddate, resample_period)
usdgbp= usdgbp[['open','high','low','close']]
usdgbp= 1/usdgbp
# usdgbp.columns = [['usdgbp_open','usdgbp_high','usdgbp_low','usdgbp_close']]

usdnok = read_pickle("USDNOK", startdate, enddate, resample_period)
usdnok= usdnok[['open','high','low','close']]
# usdnok.columns = [['usdnok_open','usdnok_high','usdnok_low','usdnok_close']]

usdsek = read_pickle("USDSEK", startdate, enddate, resample_period)
usdsek= usdsek[['open','high','low','close']]
# usdsek.columns = [['usdsek_open','usdsek_high','usdsek_low','usdsek_close']]

usdnzd = read_pickle("NZDUSD", startdate, enddate, resample_period)
usdnzd= usdnzd[['open','high','low','close']]
usdnzd= 1/usdnzd
# usdnzd.columns = [['usdnzd_open','usdnzd_high','usdnzd_low','usdnzd_close']]

In [ ]:
df = pd.concat([usdeur,usdaud,usdcad,usdchf,usdjpy,usdgbp,usdnok,usdsek],axis=1)

df = pd.concat([usdeur['close'],
                usdaud['close'],
                usdcad['close'],
                usdchf['close'],
                usdjpy['close'],
                usdgbp['close'],
                usdnok['close'],
                usdsek['close'], 
                usdnzd['close']], axis=1)

df.index = df.index.to_pydatetime()
df.sort_index(inplace=True)
df = df[df.index.dayofweek < 5]

fx_c = df.dropna()
fx_c.columns = [['usdeur_close',
                 'usdaud_close',
                 'usdcad_close',
                 'usdchf_close',
                 'usdjpy_close',
                 'usdgbp_close',
                 'usdnok_close',
                 'usdsek_close',
                 'usdnzd_close']]

fx_c_rets = fx_c.pct_change()

In [ ]:
# original model, no limit orders, assumes no transaction costs

fx_c_ranks = fx_c_rets.rank(axis=1, ascending=False)
df_long_signal = np.sign(fx_c_ranks[fx_c_ranks<=1])
df_short_signal = np.sign(fx_c_ranks[fx_c_ranks>=9])*(-1)
long_rets = df_long_signal.shift(1) * fx_c_rets*(-1)
short_rets = df_short_signal.shift(1) * fx_c_rets*(-1)
long_rets_sum = long_rets.mean(axis=1)
short_rets_sum = short_rets.mean(axis=1)
mr_pnl = (long_rets_sum + short_rets_sum) / 2

print(np.mean(mr_pnl)*100)

mr_cum_pnl = mr_pnl.cumsum()
mr_cum_pnl.plot()
plt.show()

In [ ]:
long_rets['usdeur_close'].cumsum().plot()

# Re run everything with 1 minute data, then create 15 min OHLC bars

In [ ]:
# reload data for 1 min 
instrument = ""
startdate = '2015-01-01'
enddate = '2018-12-31'
resample_period = "1t"


usdeur = read_pickle("EURUSD", startdate, enddate, resample_period)
usdeur= usdeur[['open','high','low','close']]
usdeur= 1/usdeur
# usdeur.columns = [['usdeur_open','usdeur_high','usdeur_low','usdeur_close']]

usdaud = read_pickle("AUDUSD", startdate, enddate, resample_period)
usdaud= usdaud[['open','high','low','close']]
usdaud= 1/usdaud
# usdaud.columns = [['usdaud_open','usdaud_high','usdaud_low','usdaud_close']]

usdcad = read_pickle("USDCAD", startdate, enddate, resample_period)
usdcad= usdcad[['open','high','low','close']]
# usdcad.columns = [['usdcad_open','usdcad_high','usdcad_low','usdcad_close']]

usdchf = read_pickle("USDCHF", startdate, enddate, resample_period)
usdchf= usdchf[['open','high','low','close']]
# usdchf.columns = [['usdchf_open','usdchf_high','usdchf_low','usdchf_close']]

usdjpy = read_pickle("USDJPY", startdate, enddate, resample_period)
usdjpy= usdjpy[['open','high','low','close']]
# usdjpy.columns = [['usdjpy_open','usdjpy_high','usdjpy_low','usdjpy_close']]

usdgbp = read_pickle("GBPUSD", startdate, enddate, resample_period)
usdgbp= usdgbp[['open','high','low','close']]
usdgbp= 1/usdgbp
# usdgbp.columns = [['usdgbp_open','usdgbp_high','usdgbp_low','usdgbp_close']]

usdnok = read_pickle("USDNOK", startdate, enddate, resample_period)
usdnok= usdnok[['open','high','low','close']]
# usdnok.columns = [['usdnok_open','usdnok_high','usdnok_low','usdnok_close']]

usdsek = read_pickle("USDSEK", startdate, enddate, resample_period)
usdsek= usdsek[['open','high','low','close']]
# usdsek.columns = [['usdsek_open','usdsek_high','usdsek_low','usdsek_close']]

usdnzd = read_pickle("NZDUSD", startdate, enddate, resample_period)
usdnzd= usdnzd[['open','high','low','close']]
usdnzd= 1/usdnzd
# usdnzd.columns = [['usdnzd_open','usdnzd_high','usdnzd_low','usdnzd_close']]

In [ ]:
# create OHLC 15 min bars

usdeur_15m = usdeur.resample("15t").agg({'open': 'first',
                                          'high': 'max',
                                          'low': 'min',
                                          'close': 'last'})

usdaud_15m = usdaud.resample("15t").agg({'open': 'first',
                                          'high': 'max',
                                          'low': 'min',
                                          'close': 'last'})

usdcad_15m = usdcad.resample("15t").agg({'open': 'first',
                                          'high': 'max',
                                          'low': 'min',
                                          'close': 'last'})

In [ ]:
# total bid:ask spread per currency pair
spread = 0.00004
halfspread = spread / 2

In [ ]:
# calculate fill levels and whether these have been hit
usdeur_15m['ret'] = usdeur_15m['close'].pct_change()
usdeur_15m['c_to_l'] = (usdeur_15m['low'].shift(1) / usdeur_15m['close'] - 1)
usdeur_15m['c_to_h'] = (usdeur_15m['high'].shift(1) / usdeur_15m['close'] -1)
usdeur_15m['l_to_h'] = (usdeur_15m['high'] / usdeur_15m['low'] -1) * 100

usdeur_15m['long_fill'] = np.where(usdeur_15m['c_to_l'] < -spread, 1, 0)
usdeur_15m['long_entry'] = usdeur_15m['close'].shift(1)*(1-spread)
usdeur_15m['short_fill'] = np.where(usdeur_15m['c_to_h'] > spread, 1, 0)

# is there something to gain from looking at current bar h/l to next bar h/l ? 
# I.e. if current low > xx buy til high?


In [ ]:
# adjust the signal to account for being filled 
usdeur_15m['long_sig'] = df_long_signal['usdeur_close']
usdeur_15m['short_sig'] = df_short_signal['usdeur_close']
usdeur_15m['long_sig_2'] = usdeur_15m['long_sig'] * usdeur_15m['long_fill']
usdeur_15m['short_sig_2'] = usdeur_15m['short_sig'] * usdeur_15m['short_fill']
usdeur_15m['longpnl'] = usdeur_15m['long_sig_2'].shift(1) * (usdeur_15m['ret'] - halfspread) * (-1)
usdeur_15m['shortpnl'] = usdeur_15m['short_sig_2'].shift(1) * (usdeur_15m['ret'] + halfspread) * (-1)
usdeur_15m['longpnl'].cumsum().plot()

In [ ]:
# calculate probability of fill
eur_prob_long_fill = usdeur_15m['long_fill'].sum() / len(usdeur_15m['long_fill'])
eur_prob_short_fill = usdeur_15m['short_fill'].sum() / len(usdeur_15m['short_fill'])
eur_prob_fill = (eur_prob_long_fill + eur_prob_short_fill) / 2
# calculate yield per trade
eur_yield = (((np.mean(usdeur_15m['longpnl'])*100) + (np.mean(usdeur_15m['longpnl'])*100)) / 2) 
eur_yield 

In [ ]:
# calculate pnl potential 
trades_per_hour = 4

prob_fill = eur_prob_fill
trade_yield = eur_yield

trades_per_day = 24 * trades_per_hour * prob_fill 
daily_yield = (trades_per_day * trade_yield) / 100
daily_yield

In [ ]:
# annual pnl with 10m of trading capital
daily_yield * 10000000

# High frequency strategy description

Bid:ask spread is about 0.4 bps per instrument. 

If we get long signal, we bid at 0.4 bps below the mid. 

We are filled if the low is lower than this bid. 

Exit is a market order, therefore the spread we capture is half of the bid:ask spread, i.e. 0.4 bps / 2 = 0.2 bps.

To test. We get signal at 1200 and we then walk through each minute looking for the entry price. Then we continue to walk forward looking for a exit price, i.e. we try to offer out our position. If not filled, we market order. 








0.0006 and / 3 = 0.004428552285332181


In [ ]:
# export some data to csv to manually check results
usdeur_15m.loc["2017-02-03" : "2017-02-10"].to_csv("test.csv")